# Data cleaning

In [149]:
import pandas as pd
from os.path import join
import numpy as np

## Read the data

In [150]:
src = "../data"
dst = "../data"
fname = "coded_V1.csv"
data = pd.read_csv(join(src, fname))

## Clean the data

In [151]:
# rename columns & drop empty columns
data = data\
    .rename(columns={'V=Action':'Action',
                     'V=Method':'Method',
                     'V=Discipline':'Discipline',
                     'V=Group':'Group',
                     'V=Geo-Scope':'Geo'})\
    .drop(columns=['Unnamed: 25'])

In [152]:
# list of wrong codings and their respective corrections
miscodings = {
        'policies':'openpolicies', # actions
        'openacces':'openaccess', # actions
        'interviews':'interview', # methods
        'other(casestudies)':'other', # methods
        'desk-review':'other', # methods
        'deskresearch':'other', # methods
        'bibliometric':'biblio', # methods
        'documentanalysis':'other', # methods
        'socsci':'socscie', # disciplines
        'socscie(LIS)':'socsie', # disciplines
        'socscie(psychology)':'socscie', # disciplines
        'socscie?(informationsciences)':'socscie', # disciplines
        'soscie':'socscie', # disciplines
        'socsie':'socscie', # disciplines
        'librarians':'librarian', # group
        'publishers':'publisher', # group
        'reseaercher':'researcher', # group
        'researchers':'researcher', # group
        'all':'nonspecific', # group
        'none':'nonspecific', # group
        'TW':'TWN', # geo
        'IR':'IRN', # geo
        'IN':'IND', # geo
        'Italy':'ITA', # geo
        'PK':'PAK', # geo
        'SI':'SVN', # geo
        'LA':'SA', # geo
        'missing':np.nan # general missing entry code
}

In [153]:
def split_list(list_string):
    '''
    Takes a string containing a list of encodings separated by semicolons,
    cleans the list, splits it into different entries and returns a list
    of entries. Also corrects misspellings along the way.
    '''
    if list_string != list_string: # NaN check
        return np.nan
    
    list_string = list_string.strip(';') # remove trailing ";"
    raw_entries = list_string.split(';') # split list along ";"
    
    entries = []
    for e in raw_entries:
        e = e.replace(' ', '') # remove white spaces
        if '=' in e: # remove leading column code letter
            e = e.split('=')[-1]
        if e in miscodings.keys(): # clean up wrong encodings
            e = miscodings[e]
        if e == "socsie":
            e = "socscie"
        if e != '' and e == e:
            entries.append(e)
    return entries

In [154]:
# clean up the five coded columns
cols = ['Action', 'Method', 'Discipline', 'Group', 'Geo']
for col in cols:
    data[col] = data[col].apply(split_list)

## Sanity checks

Look at the remaining categories in each coded column

In [155]:
expected_actions = {
 'openaccess',
 'opendata',
 'openeducation',
 'openevaluation',
 'openmethod',
 'openparticipation',
 'openpolicies',
 'openscience',
 'opensoftware',
 'opentools'
}
actions = []
for a in data['Action']:
    if a == a:
        actions += a
actions = set(actions)

assert actions == expected_actions

In [156]:
expected_methods = {
    'biblio',
    'documentreview',
    'interview',
    'other',
    'survey'
}

methods = []
for m in data['Method']:
    if m == m:
        methods += m
methods = set(methods)

assert methods == expected_methods

In [160]:
expected_disciplines = {
    'natscie',
    'engtech',
    'med',
    'agric',
    'socscie',
    'hum',
    'nonspecific'
}

disciplines = []
for d in data['Discipline']:
    if d == d:
        disciplines += d
disciplines = set(disciplines)

assert disciplines == expected_disciplines

In [161]:
expected_groups = {
 'researcher',
 'librarian',
 'university',
 'unisupportstaff',
 'publisher',
 'policy',
 'funder',
 'business',
 'practitioner',
 'other',
 'nonspecific'
}

groups = []
for g in data['Group']:
    if g == g:
        groups += g
groups = set(groups)

assert groups == expected_groups

In [162]:
geo = []
for g in data['Geo']:
    if g == g:
        geo += g
geo = set(geo)

In [163]:
geo

{'AF',
 'ARE',
 'ARG',
 'ARM',
 'AS',
 'AUS',
 'AUT',
 'BEL',
 'BGD',
 'BGR',
 'BOL',
 'BRA',
 'BWA',
 'CAN',
 'CHE',
 'CHI',
 'CHL',
 'CHN',
 'CND',
 'COL',
 'CRI',
 'CUB',
 'CZE',
 'DEN',
 'DEU',
 'DNK',
 'DZA',
 'EGY',
 'ESP',
 'EST',
 'ETH',
 'EU',
 'FIN',
 'FRA',
 'GBR',
 'GHA',
 'GNR',
 'GRC',
 'HKG',
 'HNK',
 'HRV',
 'HUN',
 'IDN',
 'IND',
 'IRL',
 'IRN',
 'IRQ',
 'ISR',
 'ITA',
 'JAM',
 'JPN',
 'KEN',
 'KOR',
 'LBY',
 'LKA',
 'LTU',
 'MAR',
 'MEX',
 'MYS',
 'NA',
 'NDL',
 'NGA',
 'NLD',
 'NOR',
 'NZL',
 'PAK',
 'PER',
 'PHL',
 'PNG',
 'POL',
 'PRK',
 'PRT',
 'ROU',
 'RUS',
 'SA',
 'SAU',
 'SDN',
 'SEN',
 'SGP',
 'SRB',
 'SUI',
 'SVK',
 'SVN',
 'SWE',
 'THA',
 'TUN',
 'TUR',
 'TWN',
 'TZA',
 'UGA',
 'URY',
 'USA',
 'VEN',
 'VNM',
 'ZAF',
 'ZAR',
 'ZWE',
 'nonspecific'}

## Dummy code the coded columns

In [164]:
for entries, colname in zip([actions, methods, disciplines, groups, geo], cols):
    for entry in entries:
        if entry == entry:
            data['{}_{}'.format(colname, entry)] = data[colname].apply(lambda x: entry in x if x == x else False)
            data['{}_{}'.format(colname, entry)] = data['{}_{}'.format(colname, entry)].replace({True:1, False:0})

/tmp/ipykernel_317188/2318456402.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data['{}_{}'.format(colname, entry)] = data[colname].apply(lambda x: entry in x if x == x else False)


## Export the cleaned data

In [165]:
data.to_csv(join(dst, 'V5_9_1 round coded_220216_clean.csv'), index=False)